In [3]:
import torch
import torch.nn.functional as F
from torch import nn  # for

import numpy as np  # for
import pandas as pd  # for


import plotly.graph_objects as go
import plotly.express as px
from plotly import subplots


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("cuda available")
else:
    device = torch.device("cpu")

<div dir="rtl" lang="he" xml:lang="he">


# מודלים יוצרים - _gan models_
ניתן להשתמש ברשתות נויירונים גם ליצירה של מידע הנראה כאמיתי. 
נבנה מודל כזה שיוכל ליצור כתב יד בצורה אמינה.

<div dir="rtl" lang="he" xml:lang="he">

### יבוא הדאטא
נוריד את הקבצים מאתר האינטרנט בצורה ידנית, ונלחץ אותם במקום הנכון.
הדאטא מגיע כקובץ CSV טבלה, כך שהעמודה הראשונה היא התגית _label_ וכל שאר העמודות הם העמודות של התמונה


נשתמש בפונקצייה `load_text` בשביל לטעון את הקבצים, 
לאחר מכן נמיר את הקבצים לוקטור.

In [5]:
vec = np.genfromtxt(
    "data/mnist/mnist_test/mnist_test.csv", delimiter=",", skip_header=1
)
print(
    f"""
vec shape: {vec.shape}
vec dtpye: {vec.dtype}
vec head: 
{vec[0:5]}
    """
)


vec shape: (10000, 785)
vec dtpye: float64
vec head: 
[[7. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]]
    


<div dir="rtl" lang="he" xml:lang="he">

העמודה הראשונה היא התוית _label_ 
של התמונה, וכל שאר 784 העמודות זה התמונה בשחור לבן. 

In [20]:
labels = torch.from_numpy(vec[:,0])
data = torch.from_numpy(vec[:,1:]).reshape(10000,28,28)
data_set = (data,labels)

In [22]:
px.imshow(data_set[0][5])

<div dir="rtl" lang="he" xml:lang="he">

## הרשת השופטת
נבנה את המודל שיעריך האם התמונות שניתנות לו הם אמיתיות או לא


מבנה הרשת הוא פשוט: 
יש שלוש שכבות ליניאריות 
_dence_
עם פונקציית אקטיבציה 
_ReLU_
לאחר כל שכבה כזאת מורידים חלק מהוקטורים עם 
_dropout_ 

שפר לשים לב שהשכבותהלינאירות בהתחלה מגדילים את התמונה ורק לאחר מקטינים ומחליטים מה תהיה התמונה. 

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), 784)
        output = self.model(x)
        return output

<div dir="rtl" lang="he" xml:lang="he">

## הגנרטור 
הארכיטקטורה של הגנרטור היא בדיוק ההפוכה לארכיטקטורה של המחליט 
_discriminator_/ 
ההבדל בן השכבות הוא בפונקציית האקטיבציה, 
_sigmoid_ 
מול 
_tahn_

ובשכבות ה
_dropout_ 
שאותן אין טעם לשים במודל הגנרטיבי

In [9]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 784),
            nn.Tanh(),
        )

    def forward(self, x):
        output = self.model(x)
        output = output.view(x.size(0), 1, 28, 28)
        return output


<div dir="rtl" lang="he" xml:lang="he">

# אימון המודל
נבנה אובייקט `Trainer` בשביל לאמן את המודלים שלנו

In [10]:
from typing import Union, Type, Callable,Optional,Any,List,Tuple

ModelType = nn.Module
TensorType = torch.Tensor
OptimazerType = Type[torch.optim.Optimizer]
LossFunctoinType = Callable[[TensorType,TensorType],TensorType]
SchedulerType = Type[torch.optim.lr_scheduler.LRScheduler]
VisulizeFuntoinType = Callable[[float,Optional[Any]],None]
DataType = Tuple[torch.Tensor,torch.Tensor]
PathType = str
FileType = str


In [27]:
import math  # for cilo

DATA = 0 
LABELS = 1
class TrainerGan:
    def __init__(
        self,
        discriminator: ModelType,
        generator: ModelType,
        loss_function: LossFunctoinType,
        optimizer_discriminator: OptimazerType,
        optimizer_generator:OptimazerType,
        scheduler: Optional[SchedulerType]=None,
    ):
        self.discriminator = discriminator
        self.generator = generator
        self.loss_function = loss_function
        self.optimizer_discriminator = optimizer_discriminator
        self.optimizer_generator = optimizer_generator
        self.scheduler = scheduler
        self.loss_list_D = []
        self.loss_list_G = []
        self.data = None

    def train(
        self, data_set: DataType, batch_size: int, times: int = 1, visualization: bool = True
    ):
        #creat figure to disply the loss funtion
        num_iteration = math.ceil(len(data_set) / batch_size)  # round up number of iteration
        fig = go.Figure(
            data=go.Scatter(x=torch.arange(0, num_iteration), y=self.loss_list_D)
        ).update_yaxes(range=[0, 1])
        for epoch in range(times):
            for i in range(num_iteration):
                # Data for training the discriminator
                batch_samples = data_set[DATA][i*batch_size:(i+1)*batch_size]
                batch_labels = data_set[LABELS][i*batch_size:(i+1)*batch_size]  
                latent_space_samples = torch.randn(batch_size,100)
                generated_samples = self.generator.forward(latent_space_samples)
                generated_labels = torch.zeros(batch_size,1)
                all_samples = torch.cat((batch_samples,generated_samples))
                all_labels = torch.cat((batch_labels,generated_labels))

                # Training the discriminator
                self.discriminator.zero_grad()
                output_discriminator = self.discriminator.forward(all_samples)
                loss_D = self.loss_function(output_discriminator,all_labels)
                loss_D.backward()
                self.optimizer_discriminator.step()

                # Data for training the generator
                latent_space_samples = torch.randn((batch_size, 100))


                self.generator.zero_grad()
                generated_samples = self.generator(latent_space_samples)
                output_discriminator_generated = self.discriminator(generated_samples)
                loss_G = self.loss_function(
                    output_discriminator_generated, batch_labels
                )
                loss_G.backward()
                self.optimizer_generator.step()

        


```python
trainer = Triner(model,loss,optim,sch,visul)
trainer.set_plain("first plain", )
trainer.train(data,10,shappe=Treu,lr=0.1,gamma:float=None,schler_gama:str="",plean:str="")
```

In [28]:
discriminator = Discriminator()
generator = Generator()

In [29]:
lr = 0.0001
num_epochs = 50
loss_function = nn.BCELoss()

optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [30]:
trainer = TrainerGan(discriminator,generator,loss_function,optimizer_discriminator,optimizer_generator)

In [31]:

trainer.train(data_set,4)

RuntimeError: Tensors must have same number of dimensions: got 3 and 4